# Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes

In [ ]:
# Load config
config_file = '../neat_config.ini'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        config_file)

In [ ]:
# WIP: agent_regress
agnt = multimodal_mazes.Agent(location=[5,5], channels=[1,1])
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze.generate(48)

x, y = [], []
for n in range(len(maze.mazes)):
    mz = maze.mazes[n]
    mz_goal_loc = maze.goal_locations[n]
    d_map = np.copy(maze.d_maps[n]) 
    d_map[d_map == 0.0] = np.inf
    d_map[mz_goal_loc[0], mz_goal_loc[1]] = 0.0

    # For each segment of track in the maze
    for rc in zip(np.where(mz[:,:,-1])[0], np.where(mz[:,:,-1])[1]): 
        agnt.location = np.array(rc) # set the agent's location

        if np.array_equal(agnt.location, mz_goal_loc):
            continue
        else: 
            # Record senses
            agnt.sense(mz) 
            x.append(list(agnt.channel_inputs.reshape(-1)))

            # Record best action  
            action_dists = d_map[rc[0] + agnt.actions[0], rc[1] + agnt.actions[1]]
            action_dists += np.random.rand(len(action_dists)) / 1000
            y.append(np.argmin(action_dists))
            
x = np.array(x)
y = np.array(y)

# Check label consistency
tmp = []
for n in range(x.shape[0]):
    idx = np.where((x == x[n]).all(axis=1))[0]

    if len(np.unique(y[idx])) == 2: 
        print(n)

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x, y)
print(reg.score(x, y))
print(reg.coef_)

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=True)
x_poly = poly.fit_transform(x)
reg = LinearRegression().fit(x_poly, y)
print(reg.score(x_poly, y))
print(len(reg.coef_))
print(len(reg.coef_[reg.coef_ > 0.0]))

In [ ]:
# Load test
x = np.load('../Results/test.npy')

top_agent = np.where(x['fitness'] == x['fitness'].max())

with open('../Results/test.pickle', 'rb') as file:
    genome_id, genome, channels = pickle.load(file)

print(x[top_agent])
print(genome.size())


In [ ]:
# Plotting

# Fitness 
multimodal_mazes.plot_fitness_over_generations(x, plot_species=True)

# Architecture
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D'}
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

# Maze path 
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze.generate(48)

In [ ]:
n_steps = 10
mz_n = -1
mz = maze.mazes[mz_n]
time, path = multimodal_mazes.maze_trial(mz, maze.start_locations[mz_n], maze.goal_locations[mz_n], channels, genome, config, n_steps)
multimodal_mazes.plot_path(path, mz, maze.goal_locations[mz_n], n_steps)
print(time)

In [ ]:
fitness, times, paths = [], [], []
# For each maze
for mz_n, mz in enumerate(maze.mazes):
    # Run trial
    time, path = multimodal_mazes.maze_trial(
        mz,
        maze.start_locations[mz_n],
        maze.goal_locations[mz_n],
        channels,
        genome,
        config,
        n_steps,
    )

    # Record normalised fitness
    times.append(
        1
        - (
            (time - maze.fastest_solutions[mz_n])
            / (n_steps - 1 - maze.fastest_solutions[mz_n])
        )
    )

    paths.append(
        (maze.d_maps[mz_n].max() - maze.d_maps[mz_n][path[-1][0], path[-1][1]])
        / maze.d_maps[mz_n].max()
    )

# Fitness
fitness = (np.array(times) + np.array(paths)) * 0.5

print(fitness)
print(fitness.mean())